In [42]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

sales_fact_1998_df = pd.read_csv("./hw1_data/sales_fact_1998.csv")
sales_fact_dec_1998_df = pd.read_csv("./hw1_data/sales_fact_dec_1998.csv")

time: 105 ms (started: 2022-01-03 17:11:37 +00:00)


In [43]:
transaction1 = sales_fact_1998_df.groupby(['time_id','customer_id'])['product_id'].apply(lambda x:set(x))
transaction2 = sales_fact_dec_1998_df.groupby(['time_id','customer_id'])['product_id'].apply(lambda x:set(x))

time: 760 ms (started: 2022-01-03 17:11:37 +00:00)


In [45]:
tmp = transaction1[:5].apply(lambda x:len(x)).max()
tmp
# oh_enc = OneHotEncoder()
# oh_tmp = oh_enc.fit_transform(tmp)


9

time: 4.24 ms (started: 2022-01-03 17:11:51 +00:00)
